# Rate Search

Este documento é uma tentativa de organizar o que eu venho fazendo para o TCC.

Até o momento a estratégia utilizada para atingir determinada taxa de codificação para um light field era dividí-lo em blocos e realizar uma busca por taxa alvo para cada bloco. Uma das ideias era dividir o problema em partes menores para facilitar a resolução.

Essa estratégia de fato funcionou para atingir a taxa de codificação desejada, mas como efeito colateral a qualidade da codificação ficou pior como um todo. O motivo para isso ter acontecido é que alguns blocos são mais complexos do que outros, portanto precisam de mais bits para que sejam representados com fidelidade, enquanto outros blocos são mais simples e poucos bits são o suficiente.

A partir dessa conclusão surgiu a possibilidade de buscar estratégias capazes de distribuir os lambdas para cada bloco de modo que a qualidade de codificação do light field como um todo seja melhor do que a do codificador original. Porém, ao pensar melhor sobre o problema ficou claro que a solução com um único lambda é o resultado ótimo.

Temos referências que corroboram que um único lambda é o valor ótimo.

Esse documento será um comparativo de alguns algoritmos para encontrar a taxa ótima de codificação de um light field codificado como um todo, com todos os blocos utilizando um único valor de lambda. Futuramente estes algoritmos serão implementados como um utilitário do JPLM, evitando escritas desnecessárias em arquivo, executando blocos em paralelo dentre outras estratégias para minimizar o tempo de busca.

# Configura o codificador

In [29]:
%pip install --upgrade git+https://github.com/andrefpf/py-jplm

  Cloning https://github.com/andrefpf/py-jplm to /tmp/pip-req-build-mxw2g1zw
  Running command git clone --filter=blob:none --quiet https://github.com/andrefpf/py-jplm /tmp/pip-req-build-mxw2g1zw
  Resolved https://github.com/andrefpf/py-jplm to commit 2640f726ddad0e44802f20c66f40626f1c3c1a2b
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Note: you may need to restart the kernel to use updated packages.


In [30]:
from jplm import JPLMRunner, Config
from jplm.ctc.ctc_configs import (
    CTC_LENSLET_CONFIG,
    CTC_SYNTHETIC_CONFIG,
    CTC_TAROT_CONFIG,
    CTC_SET2_CONFIG,
    CTC_POZNANLAB1_TAU_CONFIG,
)
from collections import defaultdict
import numpy as np 

def nested_dict():
    return defaultdict(nested_dict)

runner = JPLMRunner("../../parallel-jplm/bin")

In [31]:
from pathlib import Path
import re
from dataclasses import dataclass, asdict

@dataclass
class EncodingInfo:
    bpp: float
    mse: float
    bytes: int
    lf_lambda: float


MSE_REGEX = re.compile(r"(?<=Est. Total MSE:)\s*\d*(\.\d*)?")
BPP_REGEX = re.compile(r"(?<=Est. Total BPP:)\s*\d*(\.\d*)?")
BYTES_REGEX = re.compile(r"(?<=Est. bytes:)\s*\d*(\.\d*)?")


def estimate_info(input_file: Path, config: Config, lf_lambda: float):
    config = config.copy()
    config["--lambda"] = lf_lambda
    config["--number-of-threads"] = 16
    config["--show-error-estimate"] = ""
    stdout = runner.encode(input_file, "tmp.jpl", config)

    mse = MSE_REGEX.search(stdout).group()
    bpp = BPP_REGEX.search(stdout).group()
    bytes = BYTES_REGEX.search(stdout).group()

    return EncodingInfo(
        np.float128(bpp),
        np.float128(mse),
        np.float128(bytes),
        np.float128(lf_lambda),
    )
    

# Bisecção

Esse é o algoritmo mais óbvio para resolver esse tipo de problema. Ele inicia com um intervalo inicial de lambdas que determina o maior e o menor valor da busca e sucessivamente reduz este intervalo pela metade codificando no ponto médio do intervalo.

Este algoritmo sempre encontra um valor dentro do intervalo com a precisão desejada, porém pode ser necessário um número elevado de iterações.

In [32]:
def bisection_search(lf_path, config_path, target_rate, max_iter=20, precision=0.01):
    def target_found(val):
        return abs(target_rate - val) <= target_rate * precision

    steps = list()
    lambda_start = np.float128(1)
    lambda_end = np.float128(1e9)

    info = estimate_info(lf_path, config_path, lambda_start)
    steps.append(info)
    if target_found(info.bpp):
        return steps
    
    info = estimate_info(lf_path, config_path, lambda_end)
    steps.append(info)
    if target_found(info.bpp):
        return steps

    for _ in range(2, max_iter):
        lambda_half = (lambda_start + lambda_end) / 2

        info = estimate_info(lf_path, config_path, lambda_half)
        steps.append(info)
        if target_found(info.bpp):
            return steps

        if info.bpp > target_rate:
            lambda_start = lambda_half
        else:
            lambda_end = lambda_half
    
    return steps

# Hyperbolic Split

Este algoritmo explora o fato de que a curva de lambdas em relação ao bpp se parece bastante com uma função hiperbólica do tipo $\lambda = a * R ^ b$.

Para descobrir o lambda de uma taxa alvo desejada são necessárias duas codificações prévias, uma no início e outra no fim do intervalo de busca.
A codificação no início, será feita utilizando $\lambda^\text{start}$ e irá gerar uma taxa $R^\text{start}$.
A codificação no fim será feita com $\lambda^\text{end}$ e irá gerar uma taxa $R^\text{end}$.

$$
\begin{align*}
    \lambda^\text{start}_{n} &= a_n * (R^\text{start}_n) ^ {b_n} \\
    \lambda^\text{end}_{n}   &= a_n * (R^\text{end}_n) ^ {b_n} \\
    \\
    \frac{\lambda^\text{start}_n}{\lambda^\text{end}_n} &= \left(\frac{R^\text{start}_n}{R^\text{end}_n}\right) ^ {b_n}
\end{align*}
$$

Com esses valores é posível inferir as constantes `a` e `b` da seguinte forma:

$$
\begin{align*}
    b_n &= \frac{log(\lambda^\text{start}_{n}) - log(\lambda^\text{end}_{n})}
                {log(R^\text{start}_{n}) - log(R^\text{end}_{n})} \\
    \\
    a_n &= \lambda^\text{start}_{n} * (R^\text{start}_{n}) ^ {-b_n}
\end{align*}
$$


Substituindo R pela taxa alvo na função hiperbólica com os valores inferidos de `a` e `b` obtém-se uma aproximação melhor de λ. Essa aproximação irá substituir o valor de $\lambda^\text{start}$ ou de $\lambda^\text{end}$ dependendo se o resultado da codificação ficou acima ou abaixo da taxa desejada.

Se houver algum problema no cálculo, como divisões por 0, é feito um passo de busca binária, ou seja, o valor intermediário do intervalo de lambdas é escolhido.

In [33]:
def hsplit_search(lf_path, config_path, target_rate, max_iter=20, precision=0.01):
    def target_found(val):
        return abs(target_rate - val) <= target_rate * precision

    steps = list()
    lambda_start = np.float128(1)
    lambda_end = np.float128(1e9)

    info = estimate_info(lf_path, config_path, lambda_start)
    steps.append(info)
    if target_found(info.bpp):
        return steps
    rate_start = info.bpp
    
    info = estimate_info(lf_path, config_path, lambda_end)
    steps.append(info)
    if target_found(info.bpp):
        return steps
    rate_end = info.bpp

    for _ in range(2, max_iter):
        if rate_start != rate_end:
            b = (np.log(lambda_start) - np.log(lambda_end)) / (np.log(rate_start) - np.log(rate_end))
            a = lambda_start * rate_start ** (-b)
            lambda_half = a * target_rate ** b
        else:
            lambda_half = (lambda_start + lambda_end) / 2

        info = estimate_info(lf_path, config_path, lambda_half)
        steps.append(info)
        if target_found(info.bpp):
            return steps
        rate_half = info.bpp

        if info.bpp > target_rate:
            lambda_start = lambda_half
            rate_start = rate_half
        else:
            lambda_end = lambda_half
            rate_end = rate_half

    return steps

# Hyperbolic Slope

Este algoritmo também utiliza uma função hiperbólica do tipo `λ = a * (R ^ b)`.

Porém ele não necessita de dois pontos préviamente computados para executar pois ele se baseia no fato de que a derivada do erro em relação à taxa é o lambda negativo, ou seja, `dD/dR = -λ`.

Deste modo é possível inferir as constantes `a` e `b` da seguinte forma:

```dD/dR = -λ
λ = a * (R ^ b)

D = - integral(a * (R ^ b))
D = - a * (R ^ b) * R / (b + 1)
D = - λ * R / (b + 1)

b = - λ0 * R0 / D - 1
a = λ0 / (R0 ^ b)
```

Estes valores possibilitarão uma aproximação cada vez melhor de λ. Se o algoritmo for usado desta forma ele converge rapidamente para o resultado mas não atinge o valor exato que se espera. Para resolver isso o valor de lambda só se move na direção da estimativa de acordo com um fator de convergência. 

Este fator inicia em 1, mas toda vez que a busca passa direto da taxa alvo o valor cai pela metade. Isso torna a busca mais precisa conforme se aproxima do destino.
A vantagem desse algoritmo é sua simplicidade de implementação e a inexistência do overhead inicial de duas codificações nas extremidades da busca.

In [34]:
def hslope_search(lf_path, config_path, target_rate, max_iter=20, precision=0.01):
    def target_found(val):
        return abs(target_rate - val) <= target_rate * precision

    steps = list()
    factor = 1
    last_rate = 0

    a = np.float128(25)
    b = np.float128(-1.5)
    target_rate = np.float128(target_rate)
    lambda_ = a * (target_rate ** b)

    for i in range(max_iter):
        info = estimate_info(lf_path, config_path, lambda_)
        steps.append(info)
        if target_found(info.bpp):
            return steps

        b = - lambda_ * info.bpp / info.mse
        a = info.mse * info.bpp ** (-b)
        lambda_estimative = - a * b * target_rate ** (b - 1)

        if abs(lambda_ - lambda_estimative) < 1e-6:
            break

        if i > 0:
            if info.bpp < target_rate < last_rate:
                factor *= 0.5
            elif info.bpp > target_rate > last_rate:
                factor *= 0.5

        lambda_ = lambda_estimative * factor + lambda_ * (1 - factor)
        last_rate = info.bpp

    return steps

In [80]:
from time import time


def calculate_iterations(light_field_configs: dict[str, Config], target_rates: list[float]):
    data = nested_dict()
    base_lf_path = Path("../../../.pgx/")

    for lf, base_config in light_field_configs.items():
        for tr in target_rates:
            input_file = base_lf_path / lf
            print(f"Encoding {lf}, {tr}:")

            try:
                t = time()
                iterations_data = bisection_search(input_file, base_config, tr, max_iter=40)
                data[lf][tr]["bisection"] = [asdict(i) for i in iterations_data]
                print(f"bisection finished in {time() - t} seconds")
            except Exception as e:
                print(f"bisection error {e}")

            # try:
            #     t = time()
            #     iterations_data = hsplit_search(input_file, base_config, tr, max_iter=40)
            #     data[lf][tr]["hsplit"] = [asdict(i) for i in iterations_data]
            #     print(f"hyperbolic split finished in {time() - t} seconds")
            # except Exception as e:
            #     print(f"hyperbolic split error {e}")

            # try:
            #     t = time()
            #     iterations_data = hslope_search(input_file, base_config, tr, max_iter=40)
            #     data[lf][tr]["hslope"] = [asdict(i) for i in iterations_data]
            #     print(f"hyperbolic slope finished in {time() - t} seconds")
            # except Exception as e:
            #     print(f"hyperbolic slope error {e}")
    
    return data


light_field_configs = {
    "greek": CTC_SYNTHETIC_CONFIG,
    # "sideboard": CTC_SYNTHETIC_CONFIG,
    # "tarot": CTC_TAROT_CONFIG,

    # "Bikes": CTC_LENSLET_CONFIG,
    # "Danger_de_Mort": CTC_LENSLET_CONFIG,
    # "Fountain_Vincent2": CTC_LENSLET_CONFIG,
    # "Stone_Pillars_Outside": CTC_LENSLET_CONFIG,
    
    # "set2": CTC_SET2_CONFIG,
    # "poznanlab1": CTC_POZNANLAB1_TAU_CONFIG,
}

# target_rates = [0.001, 0.005, 0.02, 0.1, 0.75]
target_rates = [0.75]


iterations_report = calculate_iterations(
    light_field_configs,
    target_rates,
)

Encoding greek, 0.75:
bisection finished in 62.07552146911621 seconds


In [14]:
class CustomJsonEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.float128):
            return float(obj)
        return super().default(obj)

In [79]:
import json

with open("iterations_report.json", "w") as file:
    json.dump(iterations_report, file, indent=2, cls=CustomJsonEncoder)

In [76]:
import json

with open("iterations_report.json", "r") as file:
    iterations_report = json.load(file)

In [78]:
for lf, target_rates in iterations_report.items():
    for tr, algorithms in target_rates.items():
        algorithms["bisection"] = iterations_report_bisection[lf][float(tr)]["bisection"]

In [81]:
from pprint import pprint

pprint(iterations_report)

defaultdict(<function nested_dict at 0x7f4d85d15c60>,
            {'greek': defaultdict(<function nested_dict at 0x7f4d85d15c60>,
                                  {0.75: defaultdict(<function nested_dict at 0x7f4d85d15c60>,
                                                     {'bisection': [{'bpp': 4.85514,
                                                                     'bytes': 1.28866,
                                                                     'lf_lambda': 1.0,
                                                                     'mse': 0.733362},
                                                                    {'bpp': 0.00115741,
                                                                     'bytes': 3072.0,
                                                                     'lf_lambda': 1000000000.0,
                                                                     'mse': 13494.1},
                                                                    {'bpp

In [75]:
pprint(iterations_report_bisection["greek"]["0.75"])

defaultdict(<function nested_dict at 0x7f4d85d15c60>,
            {'bisection': defaultdict(<function nested_dict at 0x7f4d85d15c60>, {})})
